In [1]:
import getpass
import sys
import time

import numpy as np
from copy import deepcopy

from utils import calculate_perplexity, get_ptb_dataset, Vocab
from utils import ptb_iterator, sample

import tensorflow as tf
from tensorflow.python.ops.seq2seq import sequence_loss
from model import LanguageModel

In [2]:
from q3_RNNLM import RNNLM_Model

In [3]:
class Config(object):
  """Holds model hyperparams and data information.

  The config class is used to store various hyperparameters and dataset
  information parameters. Model objects are passed a Config() object at
  instantiation.
  """
  batch_size = 64
  embed_size = 50
  hidden_size = 100
  num_steps = 10
  max_epochs = 16
  early_stopping = 2
  dropout = 0.9
  lr = 0.001



In [4]:
class Self:
    vocab = Vocab()
    config = Config()

In [5]:
self = Self()

In [6]:
self.vocab.construct(get_ptb_dataset('train'))
self.encoded_train = np.array(
    [self.vocab.encode(word) for word in get_ptb_dataset('train')],
    dtype=np.int32)
self.encoded_valid = np.array(
    [self.vocab.encode(word) for word in get_ptb_dataset('valid')],
    dtype=np.int32)
self.encoded_test = np.array(
    [self.vocab.encode(word) for word in get_ptb_dataset('test')],
    dtype=np.int32)

929589.0 total words with 10000 uniques


## Restore trained model

In [4]:
config = Config()
gen_config = deepcopy(config)
gen_config.batch_size = gen_config.num_steps = 1

# We create the training model and generative model
with tf.variable_scope('RNNLM') as scope:
    model = RNNLM_Model(config)
    # This instructs gen_model to reuse the same variables as the model above
    scope.reuse_variables()
    gen_model = RNNLM_Model(gen_config)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

929589.0 total words with 10000 uniques
929589.0 total words with 10000 uniques


In [5]:
session = tf.InteractiveSession()
session.run(init)
    

In [6]:
saver.restore(session, './weights-rnnlm/ptb_rnnlm.weights')

In [7]:
test_pp = model.run_epoch(session, model.encoded_test)
print '=-=' * 5
print 'Test perplexity: {}'.format(test_pp)
print '=-=' * 5

=-==-==-==-==-=
Test perplexity: 158.574432373
=-==-==-==-==-=


In [8]:
valid_pp = model.run_epoch(session, model.encoded_valid)

In [9]:
valid_pp

174.01003

## Generator

In [68]:
state = gen_model.initial_state.eval()

In [69]:
state

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [70]:
state.shape

(1, 100)

In [71]:
starting_text = 'in palo alto'

In [72]:
tokens = [gen_model.vocab.encode(word) for word in starting_text.split()]

In [73]:
tokens

[108, 7639, 7640]

In [74]:
inputs = [tokens[-1:]]

In [75]:
inputs

[[7640]]

In [76]:

state, y_pred = session.run([gen_model.final_state, gen_model.predictions[-1]], feed_dict={
    gen_model.input_placeholder: inputs,
    gen_model.dropout_placeholder: 1,
    gen_model.initial_state: state,
})

In [77]:
y_pred

array([[  2.69278278e-06,   2.39400225e-04,   8.66865935e-06, ...,
          9.53101021e-04,   8.45851581e-07,   2.09635556e-05]])